<a href="https://colab.research.google.com/github/ogunjosam/GPP/blob/main/gpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from osgeo import osr, gdal
import glob
import pandas as pd

!pip install rioxarray
import xarray as xr
import rioxarray

In [4]:
!git clone https://github.com/ogunjosam/GPP.git

!cd GPP
!dir GPP

In [36]:
fln = glob.glob('GPP/*.tif')
coord = pd.read_csv('/content/GPP/Study Points_BP.csv')
for i,ii in enumerate(fln):
  
  ds = gdal.Open(ii)

  old_cs= osr.SpatialReference()
  old_cs.ImportFromWkt(ds.GetProjectionRef())

  # create the new coordinate system
  wgs84_wkt = """
  GEOGCS["WGS 84",
      DATUM["WGS_1984",
          SPHEROID["WGS 84",6378137,298.257223563,
              AUTHORITY["EPSG","7030"]],
          AUTHORITY["EPSG","6326"]],
      PRIMEM["Greenwich",0,
          AUTHORITY["EPSG","8901"]],
      UNIT["degree",0.01745329251994328,
          AUTHORITY["EPSG","9122"]],
      AUTHORITY["EPSG","4326"]]"""
  new_cs = osr.SpatialReference()
  new_cs .ImportFromWkt(wgs84_wkt)

  # create a transform object to convert between coordinate systems
  transform = osr.CoordinateTransformation(old_cs,new_cs) 

  width = ds.RasterXSize
  height = ds.RasterYSize
  gt = ds.GetGeoTransform()
  minx = gt[0]
  miny = gt[3] + width*gt[4] + height*gt[5] 
  maxx = gt[0] + width*gt[1] + height*gt[2]
  maxy = gt[3] 

  latlong = transform.TransformPoint(minx,maxx) 
  

(-106.6921622347572, -57.170832960928905, 0.0)

In [142]:
fln = glob.glob('GPP/*.tif')
coord = pd.read_csv('/content/GPP/Study Points_BP.csv')

for i,ii in enumerate(fln):
  nn = []
  dataarray = rioxarray.open_rasterio(ii,default_name='olusola')
  dataarray = dataarray.rio.reproject("EPSG:4326")
  dast = dataarray.to_dataset()
  dast.set_coords(("olusola"))
  dast.assign_coords(var=(dast.olusola))
  dast=dast.drop_vars(('spatial_ref','band'))
  for j,jj in enumerate(coord['y']):
    da = dast["olusola"].sel(y=coord['y'][j], x=coord['x'][j], method="nearest")
    nn.append(float(da.values))
  
  coord[str.split(str.split(ii,'.')[4],'_')[0]] = nn

coord.to_excel('final_output.xlsx')

In [131]:
da = dast["olusola"].sel(y=coord['y'][0], x=coord['x'][0], method="nearest")
[float(da.x.values),float(da.y.values),float(da.values)]

[-81.55097426071652, 45.14196544123453, 34.0]